# Tutorial for building new force field

In this tutorial, we show how to build a new force field. This tutorial should also help user understand how we arrange the modules in OpenABC. We use Mpipi force field as an example. This force field is a CG force field for protein and RNA, with each residue or nucleotide represented as 1 CG bead. 

To set up a force field, we need 2 core classes: parser and model. Parsers can analyze input molecule structures to do the coarse-graining, set up bonded interactions, etc. After parsing a molecule, the parser can be viewed as a representation of the molecule (we will call this "parsed molecule"). Model plays the role of a container. We append multiple copies of parsed molecules into model, just like putting molecules into a simulation box. Model has the methods to add forces and set up simulations. Remember that parser and model have to be compatible. 

## Define Mpipi protein and RNA parsers

We begin with setting up parsers for Mpipi model. As such model includes protein and RNA model, we need two parsers, one for protein and the other one for RNA. 

A protein parser can load atomistic or CG protein pdb files. It can automatically assign mass, charges, bonded interactions, etc. The Mpipi protein parser is defined in: openabc/forcefields/parsers/mpipi_parsers.py. In fact, due to the similarity between Mpipi protein model and HPS model, the Mpipi protein parser is very similar to HPS parser. Next, we will show some important parts in Mpipi parser code and explain. 

### Define Mpipi protein parser

We begin with defining amino acid names, mass, and charges. We also define some conversion factors to convert units. Note Mpipi sets ARG and LYS charge as 0.75, ASP and GLU charge as -0.75, HIS charge as 0.375. 

import some useful variables and conversion factors.
```
from openabc.lib.protein_lib import _amino_acids
from openabc.lib.rna_lib import _rna_nucleotides
from openabc.lib.unit_conversion import _kcal_to_kj, _angstrom_to_nm
```

define mass and charge
```
_mpipi_mass_dict = dict(ALA=71.08, ARG=156.20, ASN=114.10, ASP=115.10, CYS=103.10, 
                        GLN=128.10, GLU=129.10, GLY=57.05, HIS=137.10, ILE=113.20, 
                        LEU=113.20, LYS=128.20, MET=131.20, PHE=147.20, PRO=97.12, 
                        SER=87.08, THR=101.10, TRP=186.20, TYR=163.20, VAL=99.07, 
                        A=329.2, C=305.2, G=345.2, U=306.2)

_mpipi_charge_dict = dict(ALA=0.0, ARG=0.75, ASN=0.0, ASP=-0.75, CYS=0.0, 
                          GLN=0.0, GLU=-0.75, GLY=0.0, HIS=0.375, ILE=0.0,
                          LEU=0.0, LYS=0.75, MET=0.0, PHE=0.0, PRO=0.0,
                          SER=0.0, THR=0.0, TRP=0.0, TYR=0.0, VAL=0.0, 
                          A=-0.75, C=-0.75, G=-0.75, U=-0.75)
```

In [1]:
# let's take a look at some variables
import sys
sys.path.append('../..')
from openabc.lib.protein_lib import _amino_acids
from openabc.lib.rna_lib import _rna_nucleotides
from openabc.lib.unit_conversion import _kcal_to_kj, _angstrom_to_nm

print(_amino_acids) # print the list of amino acid names
print(_rna_nucleotides) # print the name of RNA nucleotides
print(_kcal_to_kj) # 4.184
print(_angstrom_to_nm) # 0.1


['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
['A', 'C', 'G', 'U']
4.184
0.1


Next we define the class for Mpipi protein parser. This class can parse protein and save the information of the parsed protein as a class object. 

For `__init__`, this method initialize the class object from `ca_pdb`, which is a coarse-grained protein CA model. All the atom information will be saved in `self.atoms` attribute as a pandas DataFrame. If `default_parse` is True, the molecule will be parsed with default parameters of `self.parse_mol` method (see below). 

```
class MpipiProteinParser(object):
    """
    Mpipi protein parser.
    """
    def __init__(self, ca_pdb, default_parse=True):
        """
        Initialize a protein with Mpipi model.
        
        Parameters
        ----------
        ca_pdb : str
            CA pdb file path. 
        
        default_parse : bool
            Whether to parse with default settings. 
        
        """
        self.pdb = ca_pdb
        self.atoms = helper_functions.parse_pdb(ca_pdb)
        # check if all the atoms are protein CA atoms
        assert ((self.atoms['resname'].isin(_amino_acids)).all() and self.atoms['name'].eq('CA').all())
        if default_parse:
            print('Parse molecule with default settings.')
            self.parse_mol()
```

We also define a class method that can start from atomistic pdb file, which can be directly achieved from protein structure database or structure prediction tool. This class method first do the coarse-graining by keeping only CA atoms, then initialize class object from the CA model. 

```
    @classmethod
    def from_atomistic_pdb(cls, atomistic_pdb, ca_pdb, write_TER=False, default_parse=True):
        """
        Initialize an Mpipi model protein from atomistic pdb. 
        
        Parameters
        ----------
        atomistic_pdb : str
            Atomistic pdb file path. 
        
        ca_pdb : str
            Output CA pdb file path. 
        
        write_TER : bool
            Whether to write TER between two chains. 
        
        default_parse : bool
            Whether to parse with default settings. 
        
        """
        helper_functions.atomistic_pdb_to_ca_pdb(atomistic_pdb, ca_pdb, write_TER)
        return cls(ca_pdb, default_parse)
```

`parse_mol` is the core method of `MpipiProteinParser` class. This method add all the bonded interactions (in Mpipi model, we only add bonds here as `self.protein_bonds`, as this model does not have angle or dihedral potentials), nonbonded exclusions (here only bonded atoms are excluded from nonbonded interactions), and atomic properties (the useful properties are mass and charges, we add these atomic properties to `self.atoms` as new columns). These are all the properties that are bound to the input molecule, and all these things can be duplicated correspondingly when we duplicate the molecule multiple times. On the contrary, nonbonded interactions can be determined only after we have put all the molecules into the container. As long as the atom and residue information is saved properly in self.atoms, nonbonded interactions can be correctly applied after putting all the molecules into the container. However, as protein pairs with bonded forces are usually excluded from nonbonded forces, the exclusions can be viewed as a property of the molecule, so we save the exclusions as `self.exclusions`. All the information of the parsed molecule is saved as pandas DataFrame, so users can easily modify these parameters. 

```
    def parse_mol(self, exclude12=True, mass_dict=_mpipi_mass_dict, charge_dict=_mpipi_charge_dict):
        """
        Parse molecule. 
        
        Parameters
        ----------
        exclude12 : bool
            Whether to exclude nonbonded interactions between 1-2 atoms. 
        
        mass_dict : dict
            Mass dictionary. 
        
        charge_dict : dict
            Charge dictionary. 
        
        """
        bonds = []
        n_atoms = len(self.atoms.index)
        for atom1 in range(n_atoms):
            chain1 = self.atoms.loc[atom1, 'chainID']
            if atom1 < n_atoms - 1:
                atom2 = atom1 + 1
                chain2 = self.atoms.loc[atom2, 'chainID']
                if chain1 == chain2:
                    bonds.append([atom1, atom2])
        bonds = np.array(bonds)
        self.protein_bonds = pd.DataFrame(bonds, columns=['a1', 'a2'])
        self.protein_bonds.loc[:, 'r0'] = 0.381
        self.protein_bonds.loc[:, 'k_bond'] = 2*9.6*_kcal_to_kj/(_angstrom_to_nm**2)
        if exclude12:
            self.exclusions = self.protein_bonds[['a1', 'a2']].copy()
        else:
            self.exclusions = pd.DataFrame(columns=['a1', 'a2'])
        # set mass and charge
        for i, row in self.atoms.iterrows():
            self.atoms.loc[i, 'mass'] = mass_dict[row['resname']]
            self.atoms.loc[i, 'charge'] = charge_dict[row['resname']]
```

Notably, `self.protein_bonds`, `self.exclusions` all have columns called 'a1' and 'a2'. When we combine two parsed molecules called `mol1` and `mol2`, we just need to combine `mol1.protein_bonds` and `mol2.protein_bonds`, `mol1.exclusions` and `mol2.exclusions` and update 'a1' and 'a2' columns adaptively. We will see how this works later. 

We can similarly define the parser for RNA called `MpipiRNAParser`. The workflow is similar to defining `MpipiProteinParser`. 

Next we need to define the container `MpipiModel`. This should inherit `CGModel` (`CGModel` is defined in opeanbc/forcefields/cg_model.py). `CGModel` includes many useful methods that should be inherited by any force field model class. 

Now let's take a look at `MpipiModel`. 

Initialize, importantly, please set an attribute called `self.bonded_attr_names` to include the names of all the attributes for the bonded interactions and exclusions. This is related to `append_mol` method in `CGModel`. In Mpipi, only protein bonds, RNA bonds, and exclusions are bonded attributes. By correctly specifying the bonded attribute names, the bonded attributes can be correctly combined when we append a new molecule to the container. For example, if `mol1` is already in the container, and we are appending `mol2` into it, then we need to first concatenate `mol1.atoms` and `mol2.atoms`, then we need to concatenate all the bonded attributes such as `mol1.protein_bonds` and `mol2.protein_bonds`, `mol1.rna_bonds` and `mol2.rna_bonds`, `mol1.exclusions` and `mol2.exclusions`, and change atom indices adaptively. For example, if `mol1` and `mol2` both have 5 atoms, then atom 2 in `mol2` should have index 6 after being appended to the container (atom indices start from 0). See the code of `append_mol` in `CGModel` for more details. 
```
class MpipiModel(CGModel):
    """
    The class for Mpipi protein and RNA models. 
    This class inherits CGModel class. 
    We follow the parameters provided in the Mpipi LAMMPS input file. 
    """
    def __init__(self):
        """
        Initialize. 
        
        """
        self.atoms = None
        self.bonded_attr_names = ['protein_bonds', 'rna_bonds', 'exclusions']
```

Add protein and RNA bonds. 
```
    def add_protein_bonds(self, force_group=1):
        """
        Add protein bonds. 
        
        Parameters
        ----------
        force_group : int
            Force group. 
        
        """
        print('Add protein bonds.')
        force = functional_terms.harmonic_bond_term(self.protein_bonds, self.use_pbc, force_group)
        self.system.addForce(force)
    
    def add_rna_bonds(self, force_group=2):
        """
        Add RNA bonds. 
        
        Parameters
        ----------
        force_group : int
            Force group. 
        
        """
        print('Add RNA bonds.')
        force = functional_terms.harmonic_bond_term(self.rna_bonds, self.use_pbc, force_group)
        self.system.addForce(force)
```

Add nonbonded contacts. This potential has the Wang-Frenkel functional form. The parameters are saved in openabc/forcefields/parameters/Mpipi_parameters.csv. 
```
    def add_contacts(self, cutoff_to_sigma_ratio=3, force_group=3):
        """
        Add nonbonded contacts, which is of Wang-Frenkel functional form. 
        
        Parameters
        ----------
        
        """
        print('Add nonbonded contacts.')
        atom_types = []
        for i, row in self.atoms.iterrows():
            if (row['resname'] in (_amino_acids + _rna_nucleotides)) and (row['name'] in ['CA', 'RN']):
                atom_types.append((_amino_acids + _rna_nucleotides).index(row['resname']))
            else:
                sys.exit('Error: atom type cannot recognize.')
        epsilon_wf_map = np.zeros((len(_amino_acids + _rna_nucleotides), len(_amino_acids + _rna_nucleotides)))
        sigma_wf_map = np.zeros((len(_amino_acids + _rna_nucleotides), len(_amino_acids + _rna_nucleotides)))
        mu_wf_map = np.zeros((len(_amino_acids + _rna_nucleotides), len(_amino_acids + _rna_nucleotides)))
        nu_wf_map = np.zeros((len(_amino_acids + _rna_nucleotides), len(_amino_acids + _rna_nucleotides)))
        # in df_Mpipi_parameters, RNA nucleotide names are RNA_A, RNA_C, RNA_G, RNA_U
        df_Mpipi_parameters = pd.read_csv(f'{__location__}/parameters/Mpipi_parameters.csv')
        _ext_rna_nucleotides = [f'RNA_{x}' for x in _rna_nucleotides]
        for i, row in df_Mpipi_parameters.iterrows():
            a1 = (_amino_acids + _ext_rna_nucleotides).index(row['atom_type1'])
            a2 = (_amino_acids + _ext_rna_nucleotides).index(row['atom_type2'])
            epsilon_wf_map[a1, a2] = float(row['epsilon'])
            epsilon_wf_map[a2, a1] = float(row['epsilon'])
            sigma_wf_map[a1, a2] = float(row['sigma'])
            sigma_wf_map[a2, a1] = float(row['sigma'])
            mu_wf_map[a1, a2] = float(row['mu'])
            mu_wf_map[a2, a1] = float(row['mu'])
            nu_wf_map[a1, a2] = float(row['nu'])
            nu_wf_map[a2, a1] = float(row['nu'])
        force = functional_terms.wang_frenkel_term(atom_types, self.exclusions, self.use_pbc, epsilon_wf_map, 
                                                  sigma_wf_map, mu_wf_map, nu_wf_map, cutoff_to_sigma_ratio, 
                                                  force_group)
        self.system.addForce(force)
```

Finally add the Debye-Huckel electrostatic interaction. 
```
    def add_dh_elec(self, ldby=(1/1.26)*unit.nanometer, dielectric_water=80.0, cutoff=3.5*unit.nanometer, 
                    force_group=4):
        """
        Add Debye-Huckel electrostatic interactions. 
        
        Parameters
        ----------
        ldby : Quantity
            Debye length. 
        
        dielectric_water : float or int
            Dielectric constant of water. 
        
        cutoff : Quantity
            Cutoff distance. 
        
        force_group : int
            Force group. 
        
        """
        print('Add Debye-Huckel electrostatic interactions.')
        print(f'Set Debye length as {ldby.value_in_unit(unit.nanometer)} nm.')
        print(f'Set water dielectric as {dielectric_water}.')
        charges = self.atoms['charge'].tolist()
        force = functional_terms.dh_elec_term(charges, self.exclusions, self.use_pbc, ldby, dielectric_water, 
                                              cutoff, force_group)
        self.system.addForce(force)
```

Now we have managed to write up a new force field! We should try to compare our results with some benchmarks to ensure our implementation is correct. The validation of Mpipi force field is in ../../tests/check-Mpipi. We compare the openmm energy of a system composed of 1 polyR, 1 polyK, and 2 polyU to the LAMMPS energy. This validation ensures our implementation is correct. 